In [ ]:
# ==============================================================================
# Script:           process_methylation.py
# Purpose:          Entry-point to download and preprocess DNA methylation data
#                   for a specified project
# Author:           Sophia Li
# Affiliation:      CCG Lab, Princess Margaret Cancer Center, UHN, UofT
# Date:             11/18/2025
#Okay
# Configurations:   methylation_preproc.yaml
#
# Notes:            Checks to see if data needs to be downloaded by simply 
#                   checking if the raw data folder is empty, does not support
#                   partial downloads
# ==============================================================================

In [2]:
import pandas as pd
from pathlib import Path
import argparse
import os

from MethylCDM.utils.utils import init_environment, load_config, resolve_path
from MethylCDM.data.load_methylation import (
    download_methylation, 
    clean_methylation_data
)
from MethylCDM.preprocessing.process_methylation import process_methylation

from MethylCDM.constants import (
    RAW_METHYLATION_DIR, 
    PROCESSED_METHYLATION_DIR,
    METADATA_METHYLATION_DIR
)

%load_ext autoreload
%autoreload 2

In [5]:
args = {
    "project": "TCGA-COAD",
    "config_pipeline": "pipeline.yaml",
    "config_preproc": "methylation_preproc.yaml",
    "verbose": True
}

In [6]:
# Load the relevant configuration files 
pipeline_cfg = load_config(args['config_pipeline'])
preproc_cfg = load_config(args['config_preproc'])

# Initialize the environment for reproducible analysis
init_environment(pipeline_cfg)

In [7]:
# Check to see if DNA methylation data needs to be downloaded
raw_data_dir = preproc_cfg.get('download', {}).get('raw_data_dir', '')
raw_data_dir = resolve_path(raw_data_dir, RAW_METHYLATION_DIR)
project_raw_dir = os.path.join(raw_data_dir, args['project'])
Path(project_raw_dir).mkdir(parents = True, exist_ok = True)

In [ ]:
download_methylation(args['project'], preproc_cfg, args['verbose'])

In [ ]:
clean_methylation_data(project_raw_dir, args['verbose'])

In [15]:
# -----| Data Preprocessing |-----
metadata_dir = preproc_cfg.get('download', {}).get('metadata_dir', '')
metadata_dir = resolve_path(metadata_dir, METADATA_METHYLATION_DIR)
project_metadata = os.path.join(metadata_dir, args['project'],
                                f"{args['project']}_metadata.csv")
metadata = pd.read_csv(project_metadata)

In [16]:
# Preprocess the CpG matrix, outputting a gene-level matrix (AnnData)
gene_matrix = process_methylation(
    args['project'], metadata, preproc_cfg, args['verbose']
)

Beginning methylation preprocessing of project TCGA-COAD
Successfully performed probe quality control
Successfully performed sample quality control
Successfully imputed missing values
Successfully aggregated gene-level beta values
Successfully clipped gene-level beta values
Completed processing data for TCGA project TCGA-COAD


In [17]:
import anndata

proc_data_dir = (preproc_cfg.get('preprocess', {})
                            .get('processed_data_dir', ''))
proc_data_dir = resolve_path(proc_data_dir, PROCESSED_METHYLATION_DIR)
Path(proc_data_dir).mkdir(parents = True, exist_ok = True)

proc_file = os.path.join(proc_data_dir,
                         f"{args['project']}_gene_matrix.h5ad")

anndata.settings.allow_write_nullable_strings = True
gene_matrix.write_h5ad(proc_file, compression = "gzip")